In [1]:
from functools import partial
from importlib import reload
import unittest

from silq.pulses import PulseSequence, DCPulse, TriggerPulse, SinePulse
from silq.instrument_interfaces import get_instrument_interface
from silq.meta_instruments.chip import Chip
from silq.meta_instruments.layout import Layout
from silq.meta_instruments.mock_instruments import MockArbStudio, MockPulseBlaster

from qcodes import Instrument

<IPython.core.display.Javascript object>

In [2]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.restart_run_all();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [3]:
import qcodes as qc
qc.show_subprocess_widget()

In [4]:
class TestPulseSequence(unittest.TestCase):
    def setUp(self):
        self.pulse_sequence = PulseSequence()
        
    def test_add_remove_pulse(self):
        pulse = DCPulse(name='dc', amplitude=1.5, duration=10, t_start=0)
        self.pulse_sequence.add(pulse)
        self.assertIn(pulse, self.pulse_sequence)

        # Remove pulses
        self.pulse_sequence.clear()
        self.assertEqual(len(self.pulse_sequence.pulses), 0)
        
    def test_sort(self):
        pulse1 = DCPulse(name='dc1', amplitude=1.5, duration=10, t_start=1)
        pulse2 = DCPulse(name='dc2', amplitude=1.5, duration=10, t_start=0)
        self.pulse_sequence.add(pulse1)
        self.pulse_sequence.add(pulse2)
        self.assertEqual(pulse2, self.pulse_sequence[0])

In [5]:
class TestArbStudio(unittest.TestCase):
    def setUp(self):
        self.pulse_sequence = PulseSequence()
        self.arbstudio = MockArbStudio(name='mock_arbstudio')
        self.arbstudio_interface = get_instrument_interface(self.arbstudio)
        
    def tearDown(self):
        self.arbstudio.close()
        self.arbstudio_interface.close()
    
    def test_pulse_implementation(self):
        sine_pulse = SinePulse(t_start=0, duration=10, frequency=1e6, amplitude=1)
        self.assertIsNone(self.arbstudio_interface.get_pulse_implementation(sine_pulse))
        
        DC_pulse = DCPulse(t_start=0, duration=10, amplitude=1)
        self.assertIsNotNone(self.arbstudio_interface.get_pulse_implementation(DC_pulse))
        DC_pulse.amplitude=3
        self.assertIsNone(self.arbstudio_interface.get_pulse_implementation(DC_pulse))
        
    def test_ELR_programming(self):
        empty_pulse = DCPulse(name='empty', t_start=0, duration=10, amplitude=1.5)
        load_pulse = DCPulse(name='load', t_start=10, duration=10, amplitude=-1.5)
        read_pulse = DCPulse(name='read', t_start=20, duration=10, amplitude=0)
        pulses = [empty_pulse, load_pulse, read_pulse]
        for pulse in pulses:
            self.arbstudio_interface.pulse_sequence(('add', pulse))
        self.assertEqual(pulses, self.arbstudio_interface.pulse_sequence().pulses)

In [6]:
class TestLayout(unittest.TestCase):
    def setUp(self):
        self.arbstudio = MockArbStudio(name='arbstudio')
        self.pulseblaster = MockPulseBlaster(name='pulseblaster')
        self.chip = Chip(name='chip')
        
        self.instruments = [self.arbstudio, self.pulseblaster, self.chip]
        self.instrument_interfaces = {instrument.name: get_instrument_interface(instrument) 
                                      for instrument in self.instruments}
        self.layout = Layout(name='Layout',
                             instrument_interfaces = list(self.instrument_interfaces.values()),
                             server_name='layout')
        self.instruments += [self.layout]
        # print('Layout instruments: {}'.format(self.layout.instruments()))
        
        # instruments that need to be closed during tear down
        
    def tearDown(self):
        for instrument in self.instruments:
            instrument.close()
        for instrument_interface in self.instrument_interfaces.values():
            instrument_interface.close()

    def test_connections(self):
        self.layout.add_connection(output='arbstudio.ch1',
                                   input='device.TGAC', default=True)
        self.layout.add_connection(output='arbstudio.ch2',
                                   input='device.DF')
        connections = self.layout.get_connections()
        self.assertEqual(len(connections), 2)
        
        connections = self.layout.get_connections(input_channel='DF')
        self.assertEqual(len(connections), 1)
        self.assertEqual(connections[0].input['channel'], 'DF')
        
        DC_pulse = DCPulse(t_start=0, duration=10, amplitude=1)
        connection = self.layout.get_pulse_connection(DC_pulse)
        self.assertEqual(connection.output['channel'], 'ch1')

    def test_pulse_implementation(self):
        sine_pulse = SinePulse(t_start=0, duration=10, frequency=1e6, amplitude=1)
        with self.assertRaises(Exception):
            pulse_instrument = self.layout.get_pulse_instrument_name(sine_pulse)
            
        DC_pulse = DCPulse(t_start=0, duration=10, amplitude=1)
        pulse_instrument = self.layout.get_pulse_instrument(DC_pulse)
        self.assertEqual(pulse_instrument, 'arbstudio')
        
        DC_pulse2 = DCPulse(t_start=0, duration=10, amplitude=4)
        with self.assertRaises(Exception):
            pulse_instrument = self.layout.get_pulse_instrument(DC_pulse2)

    def test_setup_pulses(self):
        self.layout.trigger_instrument('arbstudio')
        self.layout.add_connection(output='arbstudio.ch1',
                                   input='device.TGAC')
        self.layout.add_connection(output='arbstudio.ch2',
                                   input='device.DF', default=True)
        
        pulse_sequence = PulseSequence()
        empty_pulse = DCPulse(name='empty', t_start=0, duration=10, amplitude=1.5)
        load_pulse = DCPulse(name='load', t_start=10, duration=10, amplitude=-1.5)
        read_pulse = DCPulse(name='read', t_start=20, duration=10, amplitude=0)
        pulses = [empty_pulse, load_pulse, read_pulse]
        for pulse in pulses:
            pulse_sequence.add(pulse)
            
        self.layout.target_pulse_sequence(pulse_sequence)
        waveforms = self.arbstudio.get_waveforms()
        for channel in [0,2,3]:
            self.assertEqual(len(waveforms[channel]), 0)
        self.assertEqual(len(waveforms[1]), 3)
        sequence = [self.arbstudio.ch1_sequence(),
                    self.arbstudio.ch2_sequence(),
                    self.arbstudio.ch3_sequence(),
                    self.arbstudio.ch4_sequence()]
        for channel in [0,2,3]:
            self.assertEqual(len(sequence[channel]), 0)
        self.assertEqual(len(sequence[1]), 3)

    def test_arbstudio_pulseblaster(self):
        self.layout.trigger_instrument('pulseblaster')
        self.layout.add_connection(output='arbstudio.ch1',
                                   input='device.TGAC')
        self.layout.add_connection(output='arbstudio.ch2',
                                   input='device.DF', default=True)
        
        self.layout.add_connection(output='pulseblaster.ch1',
                                   input='arbstudio.trig_in',
                                   trigger=True)
        
        trigger_connections = self.layout.get_connections(
            input_instrument='arbstudio', trigger=True)
        self.assertEqual(len(trigger_connections), 1)
        trigger_connection = trigger_connections[0]
        self.assertEqual(trigger_connection.output['instrument'], 'pulseblaster')

        pulse_sequence = PulseSequence()
        empty_pulse = DCPulse(name='empty', t_start=0, duration=10, amplitude=1.5)
        load_pulse = DCPulse(name='load', t_start=10, duration=10, amplitude=-1.5)
        read_pulse = DCPulse(name='read', t_start=20, duration=10, amplitude=0)
        pulses = [empty_pulse, load_pulse, read_pulse]
        for pulse in pulses:
            pulse_sequence.add(pulse)
            
        self.layout.target_pulse_sequence(pulse_sequence)
        
        self.assertEqual(len(self.pulseblaster.instructions()), 7)
        self.assertEqual([ins[0] for ins in self.pulseblaster.instructions()], [1,0,1,0,1,0,0])
        self.assertEqual(self.pulseblaster.instructions()[-1][2], 1)
        
        self.pulseblaster.instructions([])
        self.instrument_interfaces['pulseblaster'].ignore_first_trigger(True)
        self.layout.target_pulse_sequence(pulse_sequence)
        
        self.assertEqual(len(self.pulseblaster.instructions()), 7)
        self.assertEqual([ins[0] for ins in self.pulseblaster.instructions()], [0,0,1,0,1,0,1])

In [7]:
# Run tests    
suite_pulse_sequence = unittest.TestLoader().loadTestsFromModule(TestPulseSequence())
suite_arbstudio = unittest.TestLoader().loadTestsFromModule(TestArbStudio())
suite_layout = unittest.TestLoader().loadTestsFromModule(TestLayout())
suite = unittest.TestSuite([suite_pulse_sequence, suite_arbstudio, suite_layout])
# suite = unittest.TestSuite([suite_arbstudio])
unittest.TextTestRunner().run(suite)

....

[(0, 'continue', 0, 1000), (0, 'continue', 0, 9000000.0), (1, 'continue', 0, 1000), (0, 'continue', 0, 9000000.0), (1, 'continue', 0, 1000), (0, 'continue', 0, 9000000.0), (1, 'branch', 1, 1000)]


....
----------------------------------------------------------------------
Ran 8 tests in 10.931s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>